In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import pandas as pd
import numpy as np
from shapely.geometry import mapping
import os

# Set CRS target
target_crs = "EPSG:2056"

# File paths (replace these with actual paths)
landuse_shapefile = "/path/to/landuse_shapefile.shp"
deposition_rate_csv = "/path/to/deposition_rate.csv"
lai_raster = "/path/to/lai_raster.tif"

# Read the landuse shapefile
landuse_gdf = gpd.read_file(landuse_shapefile)
landuse_gdf = landuse_gdf.to_crs(target_crs)

# Read the deposition rate CSV
deposition_df = pd.read_csv(deposition_rate_csv, delimiter=';')

# Link the deposition rate to the shapefile based on "Art"
landuse_gdf = landuse_gdf.merge(deposition_df, on="Art")

# Process the LAI raster
with rasterio.open(lai_raster) as src:
    lai_data = src.read(1) / 1000.0  # Calculate actual LAI = cell value/1000
    # Clean the LAI data
    lai_data[lai_data < 0] = 0  # Set values less than 0 to 0
    lai_data[lai_data > 10] = 10  # Set values greater than 10 to 10

    # Extract cleaned actual LAI value to each polygon
    deposited_amounts = []
    
    for idx, row in landuse_gdf.iterrows():
        # Mask the raster with the polygon geometry
        geom = [mapping(row.geometry)]
        out_image, out_transform = mask(src, geom, crop=True)
        out_image = out_image[0]  # Extract the first band
        
        # Calculate mean LAI within the polygon
        lai_mean = np.mean(out_image[out_image > 0])  # Consider only positive values
        
        # Calculate Deposited Amount
        deposited_amount = lai_mean * row["DepositionRate"] * 1.0 * 3600  # t = 3600s, which is 1 hour
        deposited_amounts.append(deposited_amount)
        
    landuse_gdf["DepositedAmount"] = deposited_amounts

# Save the updated shapefile with DepositedAmount column
output_shapefile = "/path/to/output_shapefile.shp"
landuse_gdf.to_file(output_shapefile)

print("Deposited amount calculation completed and saved to", output_shapefile)
